<a href="https://colab.research.google.com/github/MariaBlancoGonzalez/Grupo-A1_2/blob/master/notebooks/TrenesPreprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [70]:
import pandas as pd

In [71]:
!wget https://github.com/JoseAlbertoSeco/BlablacarCarpooling_DataMining/raw/main/data/interim/trenes.csv

--2021-12-19 15:00:42--  https://github.com/JoseAlbertoSeco/BlablacarCarpooling_DataMining/raw/main/data/interim/trenes.csv
Resolving github.com (github.com)... 13.114.40.48
Connecting to github.com (github.com)|13.114.40.48|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/JoseAlbertoSeco/BlablacarCarpooling_DataMining/main/data/interim/trenes.csv [following]
--2021-12-19 15:00:42--  https://raw.githubusercontent.com/JoseAlbertoSeco/BlablacarCarpooling_DataMining/main/data/interim/trenes.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14346 (14K) [text/plain]
Saving to: ‘trenes.csv.4’

trenes.csv.4        100%[===================>]  14.01K  --.-KB/s    in 0s      

2021-12-19 15:00:43 (28.2 

In [72]:
trenes = pd.read_csv("trenes.csv") 
trenes.head(7)

,Transporte urbano,Urbano por metro,Transporte urbano regular por autobús,Transporte interurbano regular,Interurbano por autobús regular,Interurbano por ferrocarril,AVE,Fecha
0,133592,52173,81419,70872,37668,28026,1262,2021M08
1,167137,66521,100616,81502,42893,34239,1358,2021M07
2,182190,72234,109955,81422,42493,35829,1080,2021M06
3,175821,69618,106203,77522,40846,34581,847,2021M05
4,160544,63799,96746,69359,36612,31420,584,2021M04
5,168829,67151,101678,72804,38704,32831,600,2021M03
6,140016,56264,83753,57617,31087,25622,405,2021M02


In [73]:
df_blabla = pd.read_csv("/content/drive/MyDrive/Datos/blablacar_basic.csv") 
df_blabla.head(7)

,DIA,PAIS,ORIGEN,DESTINO,ASIENTOS_OFERTADOS,ASIENTOS_CONFIRMADOS,VIAJES_OFERTADOS,VIAJES_CONFIRMADOS,OFERTANTES,OFERTANTES_NUEVOS,CA_ORIGEN,CA_DESTINO
0,01/11/2017,es,A Coruña,Bilbao,8.0,7,3,3,3,0,Galicia,País Vasco
1,02/11/2017,es,A Coruña,Bilbao,2.0,1,1,1,1,0,Galicia,País Vasco
2,03/11/2017,es,A Coruña,Bilbao,7.0,4,3,2,3,0,Galicia,País Vasco
3,05/11/2017,es,A Coruña,Bilbao,13.0,2,6,2,6,0,Galicia,País Vasco
4,10/11/2017,es,A Coruña,Bilbao,10.0,3,5,2,5,0,Galicia,País Vasco
5,12/11/2017,es,A Coruña,Bilbao,7.0,4,4,2,4,0,Galicia,País Vasco
6,13/11/2017,es,A Coruña,Bilbao,4.0,1,2,1,2,0,Galicia,País Vasco


## Extraemos solo las filas que queremos

A continuación se cambia la forma en la que aparece la fecha, pasamos de 2019M01 a 2019-01

In [74]:
fecha = trenes['Fecha'].tolist()
fecha_extraer = []

for i in fecha:
  if i[0:4] == '2018' or i[0:4] == '2017' or i[0:4] == '2019':
    fecha_extraer.append(i)

In [75]:
df_auxiliar = trenes[trenes['Fecha']== fecha_extraer.pop(0)]
for i in fecha_extraer:
  aux = trenes[trenes['Fecha']== i]
  df_auxiliar = pd.concat((df_auxiliar, aux))
df_auxiliar.head(4)

,Transporte urbano,Urbano por metro,Transporte urbano regular por autobús,Transporte interurbano regular,Interurbano por autobús regular,Interurbano por ferrocarril,AVE,Fecha
20,256168,109101,147067,114328,57062,53279,1799,2019M12
21,275451,115919,159532,121477,61506,56035,1828,2019M11
22,298456,122955,175501,133766,68932,60265,2016,2019M10
23,255762,104300,151462,120128,63623,51356,1941,2019M09


In [87]:
fecha_cambiar = df_auxiliar['Fecha'].tolist()
nueva_fecha = []
for i in fecha_cambiar:
  partida = i.split('M')
  nueva_fecha.append(f'{partida[0]}-{partida[1]}')

In [77]:
# Extraemos fecha antigua
df_trenes = df_auxiliar.drop(['Fecha'], axis = 1)
df_trenes.head(4)

,Transporte urbano,Urbano por metro,Transporte urbano regular por autobús,Transporte interurbano regular,Interurbano por autobús regular,Interurbano por ferrocarril,AVE
20,256168,109101,147067,114328,57062,53279,1799
21,275451,115919,159532,121477,61506,56035,1828
22,298456,122955,175501,133766,68932,60265,2016
23,255762,104300,151462,120128,63623,51356,1941


In [78]:
# Introducimos nuevo formato de fecha con el nombre DIA para unificar con el blablar
df_trenes['DIA'] = nueva_fecha
df_trenes.head(4)

,Transporte urbano,Urbano por metro,Transporte urbano regular por autobús,Transporte interurbano regular,Interurbano por autobús regular,Interurbano por ferrocarril,AVE,DIA
20,256168,109101,147067,114328,57062,53279,1799,2019-12
21,275451,115919,159532,121477,61506,56035,1828,2019-11
22,298456,122955,175501,133766,68932,60265,2016,2019-10
23,255762,104300,151462,120128,63623,51356,1941,2019-09


## Creamos la tarjeta de datos de trenes y blablaviajes

In [79]:
df_blabla['DIA'] = pd.to_datetime(df_blabla['DIA'], errors='coerce')

Añadimos un 0 delante de los meses con un solo caracter para cambiar el tipo de fecha y que se quede igual que en los trenes

In [80]:
df_total = pd.DataFrame()
year = (df_blabla['DIA'].dt.year).tolist()
month = (df_blabla['DIA'].dt.month).tolist()
dia_blabla = []
for i in range(len(year)):
  # Un número (1, 2, 3...)
  if month[i] > 9:
    dia_blabla.append(f'{year[i]}-0{month[i]}')
  # Dos numeros (10, 11, 12)
  else:
    dia_blabla.append(f'{year[i]}-0{month[i]}')

df_total = df_blabla.drop(['DIA'], axis = 1)
df_total['DIA'] = dia_blabla
df_total

,PAIS,ORIGEN,DESTINO,ASIENTOS_OFERTADOS,ASIENTOS_CONFIRMADOS,VIAJES_OFERTADOS,VIAJES_CONFIRMADOS,OFERTANTES,OFERTANTES_NUEVOS,CA_ORIGEN,CA_DESTINO,DIA
0,es,A Coruña,Bilbao,8.0,7,3,3,3,0,Galicia,País Vasco,2017-01
1,es,A Coruña,Bilbao,2.0,1,1,1,1,0,Galicia,País Vasco,2017-02
2,es,A Coruña,Bilbao,7.0,4,3,2,3,0,Galicia,País Vasco,2017-03
3,es,A Coruña,Bilbao,13.0,2,6,2,6,0,Galicia,País Vasco,2017-05
4,es,A Coruña,Bilbao,10.0,3,5,2,5,0,Galicia,País Vasco,2017-010
...,...,...,...,...,...,...,...,...,...,...,...,...
1048486,es,Molvízar,Sant Feliu de Codines,3.0,1,1,1,1,0,Andalucía,Cataluña,2019-04
1048487,es,Torrelles de Llobregat,Villar del Arzobispo,3.0,1,1,1,1,0,Cataluña,Comunidad Valenciana,2019-03
1048488,es,Antigua,Pájara,3.0,1,1,1,1,0,Canarias,Canarias,2019-011
1048489,es,Pájara,Antigua,3.0,1,1,1,1,0,Canarias,Canarias,2019-011


In [81]:
df_sin_categoricas = df_total.drop(['ORIGEN', 'PAIS','DESTINO', 'CA_ORIGEN', 'CA_DESTINO'], axis = 1)
df_sin_categoricas.head(3)

,ASIENTOS_OFERTADOS,ASIENTOS_CONFIRMADOS,VIAJES_OFERTADOS,VIAJES_CONFIRMADOS,OFERTANTES,OFERTANTES_NUEVOS,DIA
0,8.0,7,3,3,3,0,2017-01
1,2.0,1,1,1,1,0,2017-02
2,7.0,4,3,2,3,0,2017-03


Unificamos los dos dataframes para obtener la tarjeta de datos

In [85]:
df = pd.merge(df_sin_categoricas, df_trenes, on='DIA')
df.sort_values(by='DIA', inplace=True)
df

,ASIENTOS_OFERTADOS,ASIENTOS_CONFIRMADOS,VIAJES_OFERTADOS,VIAJES_CONFIRMADOS,OFERTANTES,OFERTANTES_NUEVOS,DIA,Transporte urbano,Urbano por metro,Transporte urbano regular por autobús,Transporte interurbano regular,Interurbano por autobús regular,Interurbano por ferrocarril,AVE
0,8.0,7,3,3,3,0,2017-01,239894,98850,141045,100044,51752,45451,1457
1843,4.0,1,2,1,2,0,2017-01,239894,98850,141045,100044,51752,45451,1457
1844,1.0,1,1,1,1,0,2017-01,239894,98850,141045,100044,51752,45451,1457
1845,14.0,1,9,1,8,0,2017-01,239894,98850,141045,100044,51752,45451,1457
1846,13.0,2,6,1,6,0,2017-01,239894,98850,141045,100044,51752,45451,1457
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
764195,6.0,2,4,1,4,0,2019-09,255762,104300,151462,120128,63623,51356,1941
764196,3.0,1,4,1,4,0,2019-09,255762,104300,151462,120128,63623,51356,1941
764197,5.0,2,3,1,3,0,2019-09,255762,104300,151462,120128,63623,51356,1941
764199,10.0,1,4,1,4,0,2019-09,255762,104300,151462,120128,63623,51356,1941


Descarga de la tarjeta de datos

In [86]:
from google.colab import files

df.to_csv('df_trenes.csv')

files.download('df_trenes.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>